In [1]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()

# incognito window
chrome_options.add_argument("--incognito")

# open a webbrowser to scrape
driver = webdriver.Chrome(r'C:\Users\AKHIL\Downloads\chromedriver_win32\chromedriver.exe',options=chrome_options)

In [4]:
driver.get("https://www.yatra.com/")

In [5]:
wait = WebDriverWait(driver, 10)

In [6]:
driver.maximize_window()
driver.switch_to.window(driver.window_handles[0])

In [7]:
#this code is to just support creation of loops.
from_elm = driver.find_element_by_xpath("//li[@class='initial-tab flex1 safariFix__field']")
from_elm.click()

time.sleep(1)
airports = driver.find_elements_by_xpath("//div[@class='viewport']//div//div//li")
print(len(airports))
airports[0].click()

time.sleep(1)
airports = driver.find_elements_by_xpath("//div[@class='viewport']//div//div//li")
print(len(airports))
airports[1].click()

10
10


In [8]:
columns = ["airport_from", "airport_to", "departure_date", "departure_day", "airline", "duration",
           "stops", "departure_time", "arrival_time", "price"]
df= pd.DataFrame(columns=columns)

In [9]:
for i in range(len(airports)):
    for k in range(len(airports)):
        from_elm = driver.find_element_by_xpath("//li[@class='initial-tab flex1 safariFix__field']")
        from_elm.click()
    
        time.sleep(1)
        scroll_elm = wait.until(EC.presence_of_element_located((By.XPATH,"//ul//div[@class='viewport']")))
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_elm)
        driver.execute_script("arguments[0].scrollTop = 0", scroll_elm)
        airports1 = driver.find_elements_by_xpath("//ul//div[@class='viewport']//div//div//li")
    
        if i >= 5:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_elm)
        
        time.sleep(1)
        airport_from = airports1[i].text
        airports1[i].click()

    
        time.sleep(1)
        scroll_elm = driver.find_element_by_xpath("//div[@class='ac_results dest_ac']//ul//div[@class='viewport']")
    
        if k <= 5:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_elm)
        
        airports2 = driver.find_elements_by_xpath("//div[@class='ac_results dest_ac']//ul//div[@class='viewport']//div//div//li")
        airport_to = airports2[-(k+1)].text
        airports2[-(k+1)].click()
    
        for j in range(1,8):
            try:
                time.sleep(1)
                calender_elm = driver.find_element_by_xpath("//label[@class='inp-focus calendar-label']")
                calender_elm.click()
        
                time.sleep(1)
                date_scroll_elm =  wait.until(EC.presence_of_element_located((By.ID,"monthWrapper")))
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", date_scroll_elm)
                driver.execute_script("arguments[0].scrollTop = 0", date_scroll_elm)
    
        
                time.sleep(1)
                date_elms = driver.find_elements_by_xpath("//table[@class='days-head day-container-table']//tbody[@class='BE_flight_origin_date']//tr//td//span")
                print("len_of_dates - ",len(date_elms))
                date_elms[j+5].click()
        
                time.sleep(1)
                full_date_elm = driver.find_element_by_xpath("//section[@class='calendarComponent']//div[@class='custom-date-container']//p[@class='cutom-day-year']")
                departure_date = full_date_elm.text
        
                weekday_elm = driver.find_element_by_xpath("//section[@class='calendarComponent']//div[@class='custom-date-container']//p[@class='cutom-day-name']")
                departure_day = weekday_elm.text
        
                time.sleep(1)
                search_btn = driver.find_element_by_id("BE_flight_flsearch_btn")
                search_btn.click()
        
                time.sleep(1)
                driver.execute_script("window.scrollTo(0 ,document.body.scrollHeight);")
                for s in range(5):
                    time.sleep(1)
                    driver.execute_script("window.scrollTo(0 , 0);")
                    time.sleep(2)
                    driver.execute_script("window.scrollTo(0 ,document.body.scrollHeight);")
                time.sleep(2)
                try:
                    flights_list = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='flightItem border-shadow pr']")))
                    print("len of flights list - ",len(flights_list))
        
                    for l in range(len(flights_list)):
            
                        try:    
                            airline_elm  = driver.find_element_by_xpath("//div[@class='flightItem border-shadow pr'][{}]//div//div//div//div//div[@class='fs-13 airline-name no-pad col-8']//span".format(l+1))
                            airline = airline_elm.text
            
                            duration_elm = driver.find_element_by_xpath("//div[@class='flightItem border-shadow pr'][{}]//div[@class='stop-cont pl-13']//p".format(l+1))
                            duration = duration_elm.text
            
                            stops_elm = driver.find_element_by_xpath("//div[@class='flightItem border-shadow pr'][{}]//div[@class='stop-cont pl-13']//div//span".format(l+1))
                            stops = stops_elm.text
            
                            start_elm = driver.find_element_by_xpath("//div[@class='flightItem border-shadow pr'][{}]//div[@autom='departureTimeLabel']//div".format(l+1))
                            departure_time = start_elm.text
            
                            end_elm = driver.find_element_by_xpath("//div[@class='flightItem border-shadow pr'][{}]//p[@autom='arrivalTimeLabel']".format(l+1))
                            arrival_time = end_elm.text
            
                            fare_elm = driver.find_element_by_xpath("//div[@class='flightItem border-shadow pr'][{}]//div[@class='i-b tipsy fare-summary-tooltip fs-18']".format(l+1))
                            price = fare_elm.text
            
                            row = [airport_from, airport_to, departure_date, departure_day, airline, duration, stops, departure_time, 
                               arrival_time, price]
            
                            df = pd.concat([df,pd.DataFrame([row],columns=columns)],ignore_index = True)
                
                            time.sleep(2)
                        except:
                            pass
                    
                    time.sleep(3)
                except:
                    driver.get("https://www.yatra.com/")
                    time.sleep(3)
             
                driver.execute_script("window.history.go(-1)")
                time.sleep(5)
            
            except:
                continue
            
            
        driver.close()
        driver = webdriver.Chrome(r'C:\Users\AKHIL\Downloads\chromedriver_win32\chromedriver.exe',options=chrome_options)
        wait = WebDriverWait(driver, 10)
        driver.get("https://www.yatra.com/")
        driver.maximize_window()
        driver.switch_to.window(driver.window_handles[0])
            
        agree = driver.find_element_by_xpath("//div[@class='gdpr-headerWrapper']//button[@class='btngdpr']")
        agree.click()
        time.sleep(90)
        
        

len_of_dates -  476
len of flights list -  31
len_of_dates -  476
len of flights list -  31
len_of_dates -  476
len of flights list -  48
len_of_dates -  476
len of flights list -  43
len_of_dates -  476
len of flights list -  44
len_of_dates -  476
len of flights list -  51
len_of_dates -  476
len of flights list -  46
len_of_dates -  476
len of flights list -  20
len_of_dates -  476
len of flights list -  21
len_of_dates -  476
len of flights list -  30
len_of_dates -  476
len_of_dates -  476
len of flights list -  43
len_of_dates -  476
len of flights list -  75
len_of_dates -  476
len of flights list -  75
len_of_dates -  476
len of flights list -  75
len_of_dates -  476
len of flights list -  85
len_of_dates -  476
len of flights list -  73
len_of_dates -  476
len of flights list -  53
len_of_dates -  476
len of flights list -  85
len_of_dates -  476
len of flights list -  85
len_of_dates -  476
len of flights list -  75
len_of_dates -  476
len of flights list -  65
len_of_dates -

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=94.0.4606.71)


In [10]:
df.shape

(10905, 10)

In [11]:
df.tail()

,airport_from,airport_to,departure_date,departure_day,airline,duration,stops,departure_time,arrival_time,price
10900,Bangalore (BLR)\nKempegowda International\nIndia,Goa (GOI)\nDabolim\nIndia,12 Oct' 21,Tuesday,Vistara,18h 10m,1 Stop,19:40,13:50\n+ 1 day,"21,979"
10901,Bangalore (BLR)\nKempegowda International\nIndia,Goa (GOI)\nDabolim\nIndia,12 Oct' 21,Tuesday,Vistara,19h 55m,1 Stop,17:55,13:50\n+ 1 day,"21,979"
10902,Bangalore (BLR)\nKempegowda International\nIndia,Goa (GOI)\nDabolim\nIndia,12 Oct' 21,Tuesday,Air India,21h 20m,1 Stop,17:20,14:40\n+ 1 day,"21,979"
10903,Bangalore (BLR)\nKempegowda International\nIndia,Goa (GOI)\nDabolim\nIndia,12 Oct' 21,Tuesday,Vistara,26h 20m,1 Stop,11:30,13:50\n+ 1 day,"21,979"
10904,Bangalore (BLR)\nKempegowda International\nIndia,Goa (GOI)\nDabolim\nIndia,12 Oct' 21,Tuesday,Air India,28h 15m,1 Stop,10:25,14:40\n+ 1 day,"21,979"


In [12]:
df.sample(5)

,airport_from,airport_to,departure_date,departure_day,airline,duration,stops,departure_time,arrival_time,price
2250,New Delhi (DEL)\nIndira Gandhi International\n...,Hyderabad (HYD)\nShamshabad Rajiv Gandhi\nIndia,13 Oct' 21,Wednesday,Air Asia,5h 00m,1 Stop,11:55,16:55,"5,956"
7250,Mumbai (BOM)\nChhatrapati Shivaji Internationa...,Chennai (MAA)\nChennai International\nIndia,11 Oct' 21,Monday,IndiGo,4h 35m,1 Stop,16:30,21:05,"8,096"
6468,Mumbai (BOM)\nChhatrapati Shivaji Internationa...,Goa (GOI)\nDabolim\nIndia,15 Oct' 21,Friday,Air India,1h 30m,Non Stop,07:20,08:50,"9,303"
6110,Mumbai (BOM)\nChhatrapati Shivaji Internationa...,Goa (GOI)\nDabolim\nIndia,10 Oct' 21,Sunday,Air India,1h 20m,Non Stop,13:25,14:45,"4,263"
7533,Mumbai (BOM)\nChhatrapati Shivaji Internationa...,Chennai (MAA)\nChennai International\nIndia,15 Oct' 21,Friday,IndiGo,4h 25m,1 Stop,07:20,11:45,"8,096"


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10905 entries, 0 to 10904
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   airport_from    10905 non-null  object
 1   airport_to      10905 non-null  object
 2   departure_date  10905 non-null  object
 3   departure_day   10905 non-null  object
 4   airline         10905 non-null  object
 5   duration        10905 non-null  object
 6   stops           10905 non-null  object
 7   departure_time  10905 non-null  object
 8   arrival_time    10905 non-null  object
 9   price           10905 non-null  object
dtypes: object(10)
memory usage: 852.1+ KB


In [14]:
df.to_csv("flight_data.csv")